<a href="https://colab.research.google.com/github/Janina712/MLTSA22_JBoecher/blob/main/Other/4.%20Compute_RhythmMetrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Imports & Set-Up**

In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/3.MLU_Matched


In [6]:
reading_vowels = pd.read_excel("reading_TextGrid_comb_BG_vowelsMatched.xlsx")
interview_vowels = pd.read_excel("interview_TextGrid_comb_BG_vowelsMatched.xlsx")
reading_consonants = pd.read_excel("reading_TextGrid_comb_BG_consonantsMatched.xlsx")
interview_consonants = pd.read_excel("interview_TextGrid_comb_BG_consonantsMatched.xlsx")

In [59]:
result_reading = pd.DataFrame(columns = ["BG","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])
result_interview = pd.DataFrame(columns = ["BG","%V","DeltaC", "DeltaV", "VarcoC", "VarcoV"])

In [35]:
IDs = ['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD']

# **Define Functions**

In [8]:
def computePercV(df_vowels, df_consonants):
  numberBG = len(df_vowels["Breath.Group"].unique())
  Vsums = np.zeros(numberBG)
  Csums = np.zeros(numberBG)
  for i in range(1,numberBG+1):
    for j in range(0, len(df_vowels)):
      if df_vowels["Breath.Group"][j] == i:
        Vsums[i-1] = Vsums[i-1] + df_vowels["Duration"][j]  
  for i in range(1,numberBG+1):
    for j in range(0, len(df_consonants)):
      if df_consonants["Breath.Group"][j] == i:
        Csums[i-1] = Csums[i-1] + df_consonants["Duration"][j]
  VCsums = Vsums + Csums
  percV = Vsums/VCsums
  return(percV)

In [15]:
def computeDeltaC(df_consonants):
  grouped_vowels = df_consonants.groupby("Breath.Group")
  grouped_lists = grouped_vowels["Duration"].apply(list)
  grouped_lists = grouped_lists.reset_index()
  deltaC = np.zeros(len(grouped_lists))
  for i in range(0,len(grouped_lists)):
    deltaC[i] = np.std(grouped_lists["Duration"][i])
  return(deltaC)

In [12]:
def computeDeltaV(df_vowels):
  grouped_vowels = df_vowels.groupby("Breath.Group")
  grouped_lists = grouped_vowels["Duration"].apply(list)
  grouped_lists = grouped_lists.reset_index()
  deltaV = np.zeros(len(grouped_lists))
  for i in range(0,len(grouped_lists)):
    deltaV[i] = np.std(grouped_lists["Duration"][i])
  return(deltaV)

In [19]:
def computeVarcoC(df_consonants, deltaC):
  grouped_consonants = df_consonants.groupby("Breath.Group")
  grouped_lists = grouped_consonants["Duration"].apply(list)
  grouped_lists = grouped_lists.reset_index()
  Cavg = np.zeros(len(grouped_lists))
  for i in range(0,len(grouped_lists)):
    Cavg[i] = np.mean(grouped_lists["Duration"][i])
  VarcoC = deltaC/Cavg
  return(VarcoC)

In [18]:
def computeVarcoV(df_vowels, deltaV):
  grouped_vowels = df_vowels.groupby("Breath.Group")
  grouped_lists = grouped_vowels["Duration"].apply(list)
  grouped_lists = grouped_lists.reset_index()
  Vavg = np.zeros(len(grouped_lists))
  for i in range(0,len(grouped_lists)):
    Vavg[i] = np.mean(grouped_lists["Duration"][i])
  VarcoV = deltaV/Vavg
  return(VarcoV)

**Calculate Metrics Using Functions**

In [25]:
percV_reading = computePercV(reading_vowels, reading_consonants)
percV_interview = computePercV(interview_vowels, interview_consonants)

In [26]:
deltaC_reading = computeDeltaC(reading_consonants)
deltaC_interview = computeDeltaC(interview_consonants)

In [27]:
deltaV_reading = computeDeltaV(reading_vowels)
deltaV_interview = computeDeltaV(interview_vowels)

In [31]:
varcoC_reading = computeVarcoC(reading_consonants, deltaC_reading)
varcoC_interview = computeVarcoC(interview_consonants, deltaC_interview)

In [32]:
varcoV_reading = computeVarcoV(reading_vowels, deltaV_reading)
varcoV_interview = computeVarcoV(interview_vowels, deltaV_interview)

**Assign to DataFrame**

In [60]:
result_reading["BG"] = np.array(range(1,38))
result_reading["%V"] = percV_reading*100
result_reading["DeltaC"] = deltaC_reading
result_reading["DeltaV"] = deltaV_reading
result_reading["VarcoC"] = varcoC_reading*100
result_reading["VarcoV"] = varcoV_reading*100

In [65]:
result_interview["BG"] = np.array(range(1,226))
result_interview["%V"] = percV_interview*100
result_interview["DeltaC"] = deltaC_interview
result_interview["DeltaV"] = deltaV_interview
result_interview["VarcoC"] = varcoC_interview*100
result_interview["VarcoV"] = varcoV_interview*100